In [3]:
import pandas as pd
import geopandas as gpd
from fastkml import kml
import fiona
from shapely.geometry import Point, Polygon

from io import BytesIO,StringIO
from zipfile import ZipFile
import re,os
import numpy as np
import xml.sax, xml.sax.handler
from html.parser import HTMLParser

import kml2geojson
import json

ModuleNotFoundError: No module named 'geopandas'

### Convert KML to geojson

In [2]:
kml2geojson.convert('Current_City_of_Detroit_Neighborhoods.kml', 'Current_City_of_Detroit_Neighborhoods.json')
kml2geojson.convert('DDOT_Bus_Stops.kml', 'DDOT_Bus_Stops.json')

### Read KML

In [3]:
#fiona.drvsupport.supported_drivers['KML'] = 'rw'
#fiona.drvsupport.supported_drivers['kml'] = 'rw' 
#neighborhood = gpd.read_file('Current_City_of_Detroit_Neighborhoods.kml', driver='KML')
#buses = gpd.read_file('DDOT_Bus_Stops.kml', driver='KML')
neighborhood = gpd.read_file('Current_City_of_Detroit_Neighborhoods.json/Current_City_of_Detroit_Neighborhoods.geojson')
buses = gpd.read_file('DDOT_Bus_Stops.json/DDOT_Bus_Stops.geojson')

### Convert to dataframe

In [4]:
neighborhood_df = pd.DataFrame(neighborhood)
buses_df = pd.DataFrame(buses)

### Read CSV

In [5]:
#nb_df = pd.read_csv('Current_City_of_Detroit_Neighborhoods.csv')
bus_df = pd.read_csv('DDOT_Bus_Stops.csv')

In [6]:
bus_df

,X,Y,FID,location,position,StreetDire,Latitude,Longitude,OnStreet,AtStreet,LocationOr,Shelter,Bench,Trash,Date,StopID
0,-83.176779,42.402042,1,Fenkell & Snowden,Farside,EB,42.402042,-83.176779,Fenkell,Snowden,,0,0,0,2019/11/16 00:00:00+00,5172
1,-83.178746,42.402004,2,Fenkell & Schaefer,Farside,EB,42.402004,-83.178746,Fenkell,Schaefer,,0,0,0,2019/11/16 00:00:00+00,260
2,-83.180987,42.401980,3,Fenkell & Lesure,Farside,EB,42.401980,-83.180987,Fenkell,Lesure,,0,0,0,2019/11/16 00:00:00+00,5166
3,-83.183147,42.401917,4,Fenkell & Cruse,Farside,EB,42.401917,-83.183147,Fenkell,Cruse,,0,0,0,2019/11/16 00:00:00+00,5164
4,-83.185212,42.401852,5,Fenkell & Freeland,Farside,EB,42.401852,-83.185212,Fenkell,Freeland,,0,0,0,2019/11/16 00:00:00+00,5162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,-83.027196,42.393855,5542,St Cyril & Georgia,Farside,NB,42.393855,-83.027196,St Cyril,Georgia,,0,0,0,2019/11/16 00:00:00+00,60157
5542,-83.026947,42.395736,5543,St Cyril & Marcus,Farside,SB,42.395736,-83.026947,St Cyril,Marcus,,0,0,0,2019/11/16 00:00:00+00,60158
5543,-83.026761,42.395889,5544,St Cyril & Marcus,Nearside,NB,42.395889,-83.026761,St Cyril,Marcus,,0,0,0,2019/11/16 00:00:00+00,60159
5544,-83.025601,42.399250,5545,St Cyril & Majorie,Farside,SB,42.399250,-83.025601,St Cyril,Majorie,,0,0,0,2019/11/16 00:00:00+00,60160


### Find and add objectid to bus dataframe

In [7]:
objectid = []
for index_1, row_1 in bus_df.iterrows():
    id = ''
    ref_point = Point(row_1['X'],row_1['Y'])
    for index_2, row_2 in neighborhood_df.iterrows():
        if ref_point.within(row_2['geometry']):
            id = row_2['OBJECTID']
    objectid.append(id)
bus_df['OBJECTID'] = objectid

In [8]:
bus_df

,X,Y,FID,location,position,StreetDire,Latitude,Longitude,OnStreet,AtStreet,LocationOr,Shelter,Bench,Trash,Date,StopID,OBJECTID
0,-83.176779,42.402042,1,Fenkell & Snowden,Farside,EB,42.402042,-83.176779,Fenkell,Snowden,,0,0,0,2019/11/16 00:00:00+00,5172,205
1,-83.178746,42.402004,2,Fenkell & Schaefer,Farside,EB,42.402004,-83.178746,Fenkell,Schaefer,,0,0,0,2019/11/16 00:00:00+00,260,205
2,-83.180987,42.401980,3,Fenkell & Lesure,Farside,EB,42.401980,-83.180987,Fenkell,Lesure,,0,0,0,2019/11/16 00:00:00+00,5166,205
3,-83.183147,42.401917,4,Fenkell & Cruse,Farside,EB,42.401917,-83.183147,Fenkell,Cruse,,0,0,0,2019/11/16 00:00:00+00,5164,205
4,-83.185212,42.401852,5,Fenkell & Freeland,Farside,EB,42.401852,-83.185212,Fenkell,Freeland,,0,0,0,2019/11/16 00:00:00+00,5162,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,-83.027196,42.393855,5542,St Cyril & Georgia,Farside,NB,42.393855,-83.027196,St Cyril,Georgia,,0,0,0,2019/11/16 00:00:00+00,60157,91
5542,-83.026947,42.395736,5543,St Cyril & Marcus,Farside,SB,42.395736,-83.026947,St Cyril,Marcus,,0,0,0,2019/11/16 00:00:00+00,60158,91
5543,-83.026761,42.395889,5544,St Cyril & Marcus,Nearside,NB,42.395889,-83.026761,St Cyril,Marcus,,0,0,0,2019/11/16 00:00:00+00,60159,91
5544,-83.025601,42.399250,5545,St Cyril & Majorie,Farside,SB,42.399250,-83.025601,St Cyril,Majorie,,0,0,0,2019/11/16 00:00:00+00,60160,91


In [9]:
neighborhood_df

,fill-opacity,stroke,stroke-opacity,OBJECTID,council_district,nhood_num,nhood_name,Website,geometry
0,0.0,#ff0000,1.0,1,4,103,West End,https://theneighborhoods.org/neighborhoods/wes...,"POLYGON ((-82.99152 42.38439, -82.99249 42.384..."
1,0.0,#ff0000,1.0,2,4,105,Chandler Park,https://theneighborhoods.org/neighborhoods/cha...,"POLYGON ((-82.97585 42.38933, -82.97603 42.389..."
2,0.0,#ff0000,1.0,3,4,100,Riverbend,https://theneighborhoods.org/neighborhoods/riv...,"POLYGON ((-82.96784 42.38369, -82.96783 42.383..."
3,0.0,#ff0000,1.0,4,4,101,Jefferson Chalmers,https://theneighborhoods.org/neighborhoods/jef...,"POLYGON ((-82.95141 42.35503, -82.95292 42.354..."
4,0.0,#ff0000,1.0,5,1,29,Riverdale,https://theneighborhoods.org/neighborhoods/riv...,"POLYGON ((-83.25748 42.41460, -83.25729 42.411..."
...,...,...,...,...,...,...,...,...,...
201,0.0,#ff0000,1.0,202,2,39,Pembroke,https://theneighborhoods.org/neighborhoods/pem...,"POLYGON ((-83.16131 42.44091, -83.16889 42.440..."
202,0.0,#ff0000,1.0,203,2,42,Oak Grove,https://theneighborhoods.org/neighborhoods/oak...,"POLYGON ((-83.16121 42.43851, -83.16015 42.438..."
203,0.0,#ff0000,1.0,204,2,57,Harmony Village,https://theneighborhoods.org/neighborhoods/har...,"POLYGON ((-83.16498 42.40962, -83.16538 42.409..."
204,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...,"POLYGON ((-83.16498 42.40962, -83.16361 42.408..."


### sjoin exploration

In [10]:
neighborhood.head()

,fill-opacity,stroke,stroke-opacity,OBJECTID,council_district,nhood_num,nhood_name,Website,geometry
0,0.0,#ff0000,1.0,1,4,103,West End,https://theneighborhoods.org/neighborhoods/wes...,"POLYGON ((-82.99152 42.38439, -82.99249 42.384..."
1,0.0,#ff0000,1.0,2,4,105,Chandler Park,https://theneighborhoods.org/neighborhoods/cha...,"POLYGON ((-82.97585 42.38933, -82.97603 42.389..."
2,0.0,#ff0000,1.0,3,4,100,Riverbend,https://theneighborhoods.org/neighborhoods/riv...,"POLYGON ((-82.96784 42.38369, -82.96783 42.383..."
3,0.0,#ff0000,1.0,4,4,101,Jefferson Chalmers,https://theneighborhoods.org/neighborhoods/jef...,"POLYGON ((-82.95141 42.35503, -82.95292 42.354..."
4,0.0,#ff0000,1.0,5,1,29,Riverdale,https://theneighborhoods.org/neighborhoods/riv...,"POLYGON ((-83.25748 42.41460, -83.25729 42.411..."


In [11]:
buses.head()

,FID,location,position,StreetDire,Latitude,Longitude,OnStreet,AtStreet,LocationOr,Shelter,Bench,Trash,Date,StopID,geometry
0,1,Fenkell & Snowden,Farside,EB,42.402042,-83.176779,Fenkell,Snowden,,0,0,0,2019-11-16T00:00:00+00:00,5172,POINT (-83.17678 42.40204)
1,2,Fenkell & Schaefer,Farside,EB,42.402004,-83.178746,Fenkell,Schaefer,,0,0,0,2019-11-16T00:00:00+00:00,260,POINT (-83.17875 42.40200)
2,3,Fenkell & Lesure,Farside,EB,42.40198,-83.180987,Fenkell,Lesure,,0,0,0,2019-11-16T00:00:00+00:00,5166,POINT (-83.18099 42.40198)
3,4,Fenkell & Cruse,Farside,EB,42.401917,-83.183147,Fenkell,Cruse,,0,0,0,2019-11-16T00:00:00+00:00,5164,POINT (-83.18315 42.40192)
4,5,Fenkell & Freeland,Farside,EB,42.401852,-83.185212,Fenkell,Freeland,,0,0,0,2019-11-16T00:00:00+00:00,5162,POINT (-83.18521 42.40185)


In [12]:
bus_stops_with_neighborhood = gpd.sjoin(buses, neighborhood, how='left', op='intersects')

In [13]:
bus_stops_with_neighborhood

,FID,location,position,StreetDire,Latitude,Longitude,OnStreet,AtStreet,LocationOr,Shelter,...,geometry,index_right,fill-opacity,stroke,stroke-opacity,OBJECTID,council_district,nhood_num,nhood_name,Website
0,1,Fenkell & Snowden,Farside,EB,42.402042,-83.176779,Fenkell,Snowden,,0,...,POINT (-83.17678 42.40204),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
1,2,Fenkell & Schaefer,Farside,EB,42.402004,-83.178746,Fenkell,Schaefer,,0,...,POINT (-83.17875 42.40200),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
2,3,Fenkell & Lesure,Farside,EB,42.40198,-83.180987,Fenkell,Lesure,,0,...,POINT (-83.18099 42.40198),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
3,4,Fenkell & Cruse,Farside,EB,42.401917,-83.183147,Fenkell,Cruse,,0,...,POINT (-83.18315 42.40192),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
4,5,Fenkell & Freeland,Farside,EB,42.401852,-83.185212,Fenkell,Freeland,,0,...,POINT (-83.18521 42.40185),204.0,0.0,#ff0000,1.0,205,2,56,Bethune Community,https://theneighborhoods.org/neighborhoods/bet...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,5542,St Cyril & Georgia,Farside,NB,42.3938554,-83.0271958,St Cyril,Georgia,,0,...,POINT (-83.02720 42.39386),90.0,0.0,#ff0000,1.0,91,3,76,Airport Sub,https://theneighborhoods.org/neighborhoods/air...
5542,5543,St Cyril & Marcus,Farside,SB,42.3957358,-83.0269471,St Cyril,Marcus,,0,...,POINT (-83.02695 42.39574),90.0,0.0,#ff0000,1.0,91,3,76,Airport Sub,https://theneighborhoods.org/neighborhoods/air...
5543,5544,St Cyril & Marcus,Nearside,NB,42.395889,-83.0267607,St Cyril,Marcus,,0,...,POINT (-83.02676 42.39589),90.0,0.0,#ff0000,1.0,91,3,76,Airport Sub,https://theneighborhoods.org/neighborhoods/air...
5544,5545,St Cyril & Majorie,Farside,SB,42.3992497,-83.0256015,St Cyril,Majorie,,0,...,POINT (-83.02560 42.39925),90.0,0.0,#ff0000,1.0,91,3,76,Airport Sub,https://theneighborhoods.org/neighborhoods/air...
